In [1]:
# 必要ライブラリのインポート
import os, cv2, time, sys
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# config取得
base_dir = Path.cwd().parent.parent
config_path = base_dir / "config"
sys.path.append(str(config_path))

from config import (
    VARI_HEALTHY_MIN,
    VARI_WARN_MIN,
    VARI_WARN_SPLIT,
    MIN_VEG_RATIO_VALID,
    MIN_VEG_PIXELS_ABS,
    MIN_VEG_PIXELS_REL
)

In [3]:
# csvデータの取得
base_dir = Path(os.getcwd())
common_dir = base_dir.parent.parent / "assets" 
input_data_path = common_dir / "csv_data" / "主要被害公園エリア_座標結合済み.csv"
input_image_path = common_dir / "images" / "test_image.jpg"

df = pd.read_csv(input_data_path)
df["緯度"] = np.floor(df["緯度"] * 1e4) / 1e4
df["経度"] = np.floor(df["経度"] * 1e4) / 1e4
# df["grid_id"] = df["緯度"].map(lambda x: f"{x:.4f}") + "_" + df["経度"].map(lambda x: f"{x:.4f}")
display(df)

,park_name,経度,緯度
0,桜ヶ丘公園,139.4567,35.6390
1,桜ヶ丘公園,139.4567,35.6387
2,桜ヶ丘公園,139.4571,35.6403
3,桜ヶ丘公園,139.4571,35.6400
4,桜ヶ丘公園,139.4571,35.6398
...,...,...,...
5941,八王子霊園,139.2762,35.6599
5942,八王子霊園,139.2764,35.6607
5943,八王子霊園,139.2764,35.6601
5944,八王子霊園,139.2764,35.6599


In [4]:
# --- 追加/再定義：ユーティリティ ---

def compute_gsd(H, f_mm, sensor_w_mm, Nx):
    # GSD[m/px] と地上幅[m]
    p_m = (sensor_w_mm / 1000.0) / Nx
    f_m = f_mm / 1000.0
    GSD = (H * p_m) / f_m
    ground_width = GSD * Nx
    return GSD, ground_width

def project_latlon_to_image(df_lat, df_lon, lat0, lon0, yaw_deg, GSD, w, h):
    """平面近似+回転でpxに投影（ベクタ化）。戻り: x_img, y_img, in_image(bool)"""
    θ = np.radians(yaw_deg)
    m_per_deg_lat = 111_320.0
    m_per_deg_lon = 111_320.0 * np.cos(np.radians(lat0))
    dY = (df_lat - lat0) * m_per_deg_lat
    dX = (df_lon - lon0) * m_per_deg_lon
    xr =  dX * np.cos(θ) + dY * np.sin(θ)   # 右+
    yr = -dX * np.sin(θ) + dY * np.cos(θ)   # 上+
    cx, cy = w/2.0, h/2.0
    x_img = cx + (xr / GSD)
    y_img = cy + (-yr / GSD)
    in_image = (x_img >= 0) & (x_img < w) & (y_img >= 0) & (y_img < h)
    return x_img, y_img, in_image

def nudge_center_by_pixels(lat0, lon0, yaw_deg, GSD, dpx_left, dpx_up):
    theta = np.radians(yaw_deg)
    m_per_deg_lat = 111_320.0
    m_per_deg_lon = 111_320.0 * np.cos(np.radians(lat0))
    A = np.array([
        [ np.cos(theta)*m_per_deg_lon,  np.sin(theta)*m_per_deg_lat],
        [-np.sin(theta)*m_per_deg_lon,  np.cos(theta)*m_per_deg_lat],
    ], dtype=float)

    b = np.array([-dpx_left*GSD, dpx_up*GSD], dtype=float)
    dlon0, dlat0 = np.linalg.solve(A, b)
    return lat0 + dlat0, lon0 + dlon0


In [5]:
# データ読込＆八王子霊園に限定
df_raw = pd.read_csv(input_data_path)
df = df_raw[df_raw["park_name"].astype(str).str.contains("八王子霊園", na=False)].copy()
df = df.dropna(subset=["緯度","経度"]).drop_duplicates(subset=["緯度","経度"]).reset_index(drop=True)

# 画像とメタ
img = Image.open(input_image_path).convert("RGB")
img_rgb = np.array(img); h, w = img_rgb.shape[:2]; Nx = w

lat0 = 35.6608823
lon0 = 139.2742054
H    = 31480.0 / 2
f_mm = 82.0
sensor_w_mm = 13.2
yaw = 0

GSD, ground_width = compute_gsd(H, f_mm, sensor_w_mm, Nx)
px_20m = max(1, int(round(20.0 / GSD)))
print(f"GSD ≈ {GSD:.3f} m/px,  ground width ≈ {ground_width:.1f} m,  20m ≈ {px_20m} px")

# 微調整
lat0, lon0 = nudge_center_by_pixels(lat0, lon0, yaw, GSD, dpx_left=-90, dpx_up=-40)

# yawで再投影 & 描画（画像内のみ）
x_img, y_img, in_image = project_latlon_to_image(df["緯度"].values, df["経度"].values,
                                                 lat0, lon0, yaw, GSD, w, h)
print("points in image:", int(in_image.sum()))
print("points out of image:", int((~in_image).sum()))

# 可視化：画像内の点のみ
# dpi = 100
# fig = plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi, frameon=False)
# ax = plt.Axes(fig, [0,0,1,1]); ax.set_axis_off(); fig.add_axes(ax)
# ax.imshow(img_rgb, interpolation="none", origin="upper")
# ax.scatter(x_img[in_image], y_img[in_image], s=50, c='lime', marker='o')
# ax.set_xlim(-0.5, w-0.5); ax.set_ylim(h-0.5, -0.5)
# plt.show()

GSD ≈ 0.564 m/px,  ground width ≈ 2533.8 m,  20m ≈ 35 px
points in image: 1848
points out of image: 3


In [6]:
# === パラメータ ===
side_m  = 20.0
half_m  = side_m / 2.0
theta   = np.radians(yaw) 
id_col  = "ID" if "ID" in df.columns else ("grid_id" if "grid_id" in df.columns else None)
indices = np.where(in_image)[0]    # 画像内に入ったタイルだけ処理

def tile_polygon_corners_px(xc, yc, theta, GSD, half_m):
    offs_EN = [(-half_m, -half_m),
               ( half_m, -half_m),
               ( half_m,  half_m),
               (-half_m,  half_m)]
    c, s = np.cos(theta), np.sin(theta)
    corners = []
    for E_off, N_off in offs_EN:
        xr =  E_off*c + N_off*s  
        yr = -E_off*s + N_off*c     
        dx =  xr / GSD
        dy = -yr / GSD 
        corners.append([xc + dx, yc + dy])
    return np.array(corners, dtype=np.float32)

# in-image の点だけを使ってポリゴンをまとめて生成
indices = np.where(in_image)[0]
polys = []
for idx in indices:
    xc, yc = float(x_img[idx]), float(y_img[idx])
    poly = tile_polygon_corners_px(xc, yc, theta, GSD, half_m).astype(np.int32)
    polys.append(poly)

# 半透明の塗り
fill = np.zeros_like(img_rgb, dtype=np.uint8)
if len(polys) > 0:
    cv2.fillPoly(fill, polys, color=(0, 255, 0))

alpha = 0.18  # 透過率（見づらければ 0.1〜0.25 で調整）
blended = cv2.addWeighted(img_rgb, 1.0, fill, alpha, 0.0)

# 罫線追加
if len(polys) > 0:
    cv2.polylines(blended, polys, isClosed=True, color=(255, 255, 255),
                  thickness=4, lineType=cv2.LINE_AA)
    cv2.polylines(blended, polys, isClosed=True, color=(0, 180, 0),
                  thickness=2, lineType=cv2.LINE_AA)

# 可視化
# dpi = 100
# fig = plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi, frameon=False)
# ax = plt.Axes(fig, [0, 0, 1, 1]); ax.set_axis_off(); fig.add_axes(ax)
# ax.imshow(blended, interpolation="none", origin="upper")
# ax.set_xlim(-0.5, w-0.5); ax.set_ylim(h-0.5, -0.5)
# plt.show()

In [7]:
# 前計算（線形化 / VARI / 植生マスク）
poly_by_idx = dict(zip(indices.tolist(), polys))
arr = img_rgb.astype(np.float32) / 255.0
def srgb_to_linear(x):
    a = 0.055
    return np.where(x <= 0.04045, x/12.92, ((x + a)/(1+a))**2.4)

lin = srgb_to_linear(arr)
R, G, B = lin[...,0], lin[...,1], lin[...,2]
den  = (G + R - B)
VARI = np.clip((G - R) / np.maximum(den, 1e-6), -2.0, 2.0)
hsv  = cv2.cvtColor((arr*255).astype(np.uint8), cv2.COLOR_RGB2HSV).astype(np.float32)

H = hsv[...,0] / 180.0  # OpenCVのHは0-180 → 0-1へ正規化
S = hsv[...,1] / 255.0
V = hsv[...,2] / 255.0

eps = 1e-6
sum_rgb = arr[...,0] + arr[...,1] + arr[...,2] + eps
r = arr[...,0] / sum_rgb
g = arr[...,1] / sum_rgb
b = arr[...,2] / sum_rgb
ExG = 2*g - r - b
lab = cv2.cvtColor((arr*255).astype(np.uint8), cv2.COLOR_RGB2LAB).astype(np.float32)
a_star = lab[...,1]  # 緑 ← 128 → 赤

# 個別条件（緩めに設定：必要に応じ調整）
cond_var  = (VARI > 0.02)
cond_exg  = (ExG  > 0.03)
cond_hsv  = ((H > 0.18) & (H < 0.44) & (S > 0.15) & (V > 0.10))   # おおよそ緑の相
cond_lab  = (a_star < 122) 
# 多重基準の和集合
veg = cond_var | cond_exg | cond_hsv | cond_lab

# バウンディング矩形を作成
work_items = []
for idx, poly in poly_by_idx.items():
    x, y, w_box, h_box = cv2.boundingRect(poly)
    x1 = max(0, x); y1 = max(0, y)
    x2 = min(w, x + w_box); y2 = min(h, y + h_box)
    if x1 >= x2 or y1 >= y2:
        continue
    local_poly = (poly - np.array([[x1, y1]], dtype=np.int32))
    work_items.append({
        "idx": idx,
        "x1": x1, "y1": y1, "x2": x2, "y2": y2,
        "local_poly": local_poly
    })

# タイル1枚の処理
id_col  = "ID" if "ID" in df.columns else ("grid_id" if "grid_id" in df.columns else None)
def process_tile(item):
    idx = item["idx"]; x1=item["x1"]; y1=item["y1"]; x2=item["x2"]; y2=item["y2"]
    local_poly = item["local_poly"]

    # 局所マスク
    Hh = y2 - y1; Ww = x2 - x1
    local_mask = np.zeros((Hh, Ww), dtype=np.uint8)
    cv2.fillConvexPoly(local_mask, local_poly, 255)
    m = (local_mask > 0)

    # 局所スライス
    vari_roi = VARI[y1:y2, x1:x2]
    veg_roi  = veg [y1:y2, x1:x2]
    arr_roi  = arr [y1:y2, x1:x2, :]

    # まずは既定の植生マスク
    sel_mask = m & veg_roi
    n_mask   = int(m.sum())
    n_veg    = int(sel_mask.sum())
    veg_ratio = (n_veg / n_mask) if n_mask > 0 else 0.0
    min_count = max(MIN_VEG_PIXELS_ABS, int(MIN_VEG_PIXELS_REL * n_mask))

    # ローカルVARI分位で拾う（“相対的に緑”を救済）
    if n_veg < min_count:
        local_vari = vari_roi[m]
        local_vari = local_vari[np.isfinite(local_vari)]
        if local_vari.size > 0:
            thr = np.quantile(local_vari, 0.60)   # 上位40%
            sel_mask = m & (vari_roi > thr)
            n_veg = int(sel_mask.sum())
            veg_ratio = (n_veg / n_mask) if n_mask > 0 else 0.0

    # ローカルExG分位で拾う
    if n_veg < min_count:
        sum_rgb_roi = arr_roi[...,0] + arr_roi[...,1] + arr_roi[...,2] + 1e-6
        r_roi = arr_roi[...,0] / sum_rgb_roi
        g_roi = arr_roi[...,1] / sum_rgb_roi
        b_roi = arr_roi[...,2] / sum_rgb_roi
        exg_roi = 2*g_roi - r_roi - b_roi

        local_exg = exg_roi[m]
        local_exg = local_exg[np.isfinite(local_exg)]
        if local_exg.size > 0:
            thr_exg = np.quantile(local_exg, 0.60)
            sel_mask = m & (exg_roi > thr_exg)
            n_veg = int(sel_mask.sum())
            veg_ratio = (n_veg / n_mask) if n_mask > 0 else 0.0

    # 分類
    valid = (n_veg >= min_count) and (veg_ratio >= MIN_VEG_RATIO_VALID)

    if not valid:
        vari_med = vari_mean = vari_std = vari_min = vari_max = np.nan
        R_med = G_med = B_med = R_mean = G_mean = B_mean = np.nan
        clazz = "N/A"
    else:
        # VARI統計
        vari_sel  = vari_roi[sel_mask]
        vari_med  = float(np.nanmedian(vari_sel))
        vari_mean = float(np.nanmean(vari_sel))
        vari_std  = float(np.nanstd(vari_sel, ddof=0))
        vari_min  = float(np.nanmin(vari_sel))
        vari_max  = float(np.nanmax(vari_sel))

        # RGB統計（sRGB 0–255）
        R_s = (arr_roi[...,0][sel_mask] * 255.0)
        G_s = (arr_roi[...,1][sel_mask] * 255.0)
        B_s = (arr_roi[...,2][sel_mask] * 255.0)
        R_med = float(np.nanmedian(R_s)); G_med = float(np.nanmedian(G_s)); B_med = float(np.nanmedian(B_s))
        R_mean = float(np.nanmean(R_s));  G_mean = float(np.nanmean(G_s));  B_mean = float(np.nanmean(B_s))
        v = vari_med
        if v >= VARI_HEALTHY_MIN:
            clazz = "健康"
        elif v < VARI_WARN_MIN:
            clazz = "危険"
        elif v < VARI_WARN_SPLIT:
            clazz = "危険" 
        else:
            clazz = "要注意" 

    xc, yc = float(x_img[idx]), float(y_img[idx])
    return {
        "idx": idx,
        "x": xc, "y": yc,
        "VARI": vari_med,
        "VARI_mean": vari_mean, "VARI_std": vari_std,
        "VARI_min": vari_min,   "VARI_max": vari_max,
        "R_med": R_med, "G_med": G_med, "B_med": B_med,
        "R_mean": R_mean, "G_mean": G_mean, "B_mean": B_mean,
        "veg_ratio": veg_ratio, "n_mask": n_mask, "n_veg": n_veg,
        "class": clazz,
        "poly_global": (local_poly + np.array([[x1, y1]], dtype=np.int32))
    }

# tqdmコールバック関数
def update_eta(bar, start, done, total):
    elapsed = time.time() - start
    rate = done / elapsed if elapsed > 0 else 0.0
    rem  = (total - done) / rate if rate > 0 else float("inf")
    mm, ss = (int(rem)//60, int(rem)%60) if np.isfinite(rem) else ("--", "--")
    bar.set_postfix_str(f"ETA {mm:02d}:{ss:02d}")

# 並列実行
results = []
max_workers = max(1, min(8, os.cpu_count() or 4))
bar = tqdm(total=len(work_items), desc="VARI per tile", dynamic_ncols=True)
start = time.time()

with ThreadPoolExecutor(max_workers=max_workers) as ex:
    futs = [ex.submit(process_tile, it) for it in work_items]
    for i, fut in enumerate(as_completed(futs), 1):
        results.append(fut.result())
        bar.update(1)
        update_eta(bar, start, i, len(futs))
bar.close()

# DataFrame化・集約
results = pd.DataFrame(results).sort_values(by="idx").reset_index(drop=True)
display(results.head())
print(
    f"tiles={len(results)} | "
    f"健康={np.sum(results['class']=='健康')} | "
    f"要注意={np.sum(results['class']=='要注意')} | "
    f"危険={np.sum(results['class']=='危険')} | "
    f"N/A={np.sum(results['class']=='N/A')}"
)

# 可視化
overlay = img_rgb.copy()
for _, r in results.iterrows():
    poly = r["poly_global"].astype(np.int32)
    clazz = r["class"]
    color = (0,255,0) if clazz=="健康" else ((255,165,0) if clazz=="要注意" else (255,0,0))
    cv2.polylines(overlay, [poly], isClosed=True, color=color, thickness=2, lineType=cv2.LINE_AA)

# dpi = 100
# fig = plt.figure(figsize=(w/dpi, h/dpi), dpi=dpi, frameon=False)
# ax = plt.Axes(fig, [0,0,1,1]); ax.set_axis_off(); fig.add_axes(ax)
# ax.imshow(overlay, interpolation="none", origin="upper")
# ax.set_xlim(-0.5, w-0.5); ax.set_ylim(h-0.5, -0.5)
# plt.show()

VARI per tile:   0%|          | 0/1848 [00:00<?, ?it/s]

,idx,x,y,VARI,VARI_mean,VARI_std,VARI_min,VARI_max,R_med,G_med,B_med,R_mean,G_mean,B_mean,veg_ratio,n_mask,n_veg,class,poly_global
0,3,33.373641,1593.812263,0.042204,0.055463,0.033118,-0.019776,0.120428,174.0,179.0,156.0,172.966217,178.283035,155.141891,1.000000,1332,1332,危険,"[[15, 1611], [51, 1611], [51, 1576], [15, 1576]]"
1,4,33.582001,1629.373609,0.045946,0.077839,0.107718,-0.038145,0.914504,173.0,179.0,156.0,167.237320,173.113892,152.587799,0.994156,1369,1361,危険,"[[15, 1647], [51, 1647], [51, 1611], [15, 1611]]"
2,5,33.790360,1664.932983,0.295920,0.437962,0.388074,-0.021110,2.000000,110.5,128.0,117.0,108.784721,126.561729,117.544754,1.000000,1296,1296,健康,"[[16, 1682], [51, 1682], [51, 1647], [16, 1647]]"
3,6,68.361969,1522.484413,0.040918,0.054562,0.024280,0.020024,0.102625,173.0,179.0,154.0,172.919647,178.306061,153.948135,1.000000,1369,1369,危険,"[[50, 1540], [86, 1540], [86, 1504], [50, 1504]]"
4,7,68.570328,1558.045759,0.031870,0.032898,0.006193,0.020024,0.053582,181.0,184.0,163.0,178.914627,182.154938,162.531219,0.949700,1332,1265,危険,"[[50, 1575], [86, 1575], [86, 1540], [50, 1540]]"


tiles=1848 | 健康=566 | 要注意=570 | 危険=678 | N/A=34


In [8]:
results["緯度"] = results["idx"].map(lambda i: float(df.iloc[i]["緯度"]))
results["経度"] = results["idx"].map(lambda i: float(df.iloc[i]["経度"]))
results["park_name"]  = results["idx"].map(lambda i: str(df.iloc[i]["park_name"]))
# display(results.head())

order_cols = [
    "park_name",
    "緯度",
    "経度",
    "VARI",
    "VARI_mean",
    "VARI_std",
    "VARI_min",
    "VARI_max",
    "R_med",
    "G_med",
    "B_med",
    "R_mean",
    "G_mean",
    "B_mean",
    "veg_ratio",
    "n_mask",
    "n_veg",
]

export_df = results.loc[:, order_cols].copy()
print(export_df.shape)
display(export_df.head(20))

(1848, 17)


,park_name,緯度,経度,VARI,VARI_mean,VARI_std,VARI_min,VARI_max,R_med,G_med,B_med,R_mean,G_mean,B_mean,veg_ratio,n_mask,n_veg
0,八王子霊園,35.659925,139.260968,0.042204,0.055463,0.033118,-0.019776,0.120428,174.0,179.0,156.0,172.966217,178.283035,155.141891,1.000000,1332,1332
1,八王子霊園,35.659745,139.260969,0.045946,0.077839,0.107718,-0.038145,0.914504,173.0,179.0,156.0,167.237320,173.113892,152.587799,0.994156,1369,1361
2,八王子霊園,35.659565,139.260971,0.295920,0.437962,0.388074,-0.021110,2.000000,110.5,128.0,117.0,108.784721,126.561729,117.544754,1.000000,1296,1296
3,八王子霊園,35.660287,139.261186,0.040918,0.054562,0.024280,0.020024,0.102625,173.0,179.0,154.0,172.919647,178.306061,153.948135,1.000000,1369,1369
4,八王子霊園,35.660106,139.261188,0.031870,0.032898,0.006193,0.020024,0.053582,181.0,184.0,163.0,178.914627,182.154938,162.531219,0.949700,1332,1265
5,八王子霊園,35.659926,139.261189,0.037802,0.036288,0.004685,0.020703,0.047589,181.0,185.0,162.0,179.894058,183.534943,161.685944,0.999249,1332,1331
6,八王子霊園,35.659746,139.261190,0.040566,0.097413,0.130438,0.020008,0.876674,171.0,176.0,154.0,161.868195,168.675613,149.870560,0.977623,1296,1267
7,八王子霊園,35.660468,139.261406,0.220539,0.223840,0.060566,0.018386,0.426832,122.0,137.0,126.0,123.693207,137.990509,125.140251,1.000000,1369,1369
8,八王子霊園,35.660288,139.261407,0.037206,0.030198,0.015884,-0.010675,0.100503,181.0,184.0,162.0,179.477188,182.484146,160.928848,0.997685,1296,1293
9,八王子霊園,35.660108,139.261409,0.038107,0.036073,0.005070,0.020008,0.052084,184.0,187.0,166.0,182.346558,185.992065,164.477402,0.972994,1296,1261


In [9]:
# 保存
out_csv = common_dir / "csv_data" / "八王子霊園_航空写真からVARI値検出結果.csv"
export_df.to_csv(out_csv, index=False, encoding="utf-8-sig")
print('saved:', out_csv)

saved: c:\Users\kyous\OneDrive\デスクトップ\ハッカソン\tokyo-tree-doctor_program\tokyo-tree-doctor\ml\assets\csv_data\八王子霊園_航空写真からVARI値検出結果.csv
